In [1]:
import pandas as pd
import requests
import json
import os
import numpy as np
from scipy.stats import poisson
from scipy.stats import norm
from pprint import pprint
import numpy as np
from scipy.optimize import minimize
from scipy.stats import poisson
import warnings
import io


# Suppress divide by zero warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in log")


API_KEY = os.getenv("API_KEY")
url = 'https://data-service.beatthebookie.blog/data'
headers = {"x-api-key": API_KEY}
params = {'division':'Premier League'}
response = requests.get(url, headers=headers, params=params)
json_str = response.content.decode('utf-8')
prem_df = pd.read_json(io.StringIO(json_str))
prem_teams_25 = prem_df[prem_df["season"] == 20242025]
prem_teams_25 = pd.concat([prem_teams_25['home_team'], prem_teams_25['away_team']]).unique()

params = {'division':'Championship'}
response = requests.get(url, headers=headers, params=params)
json_str = response.content.decode('utf-8')
champ_df = pd.read_json(io.StringIO(json_str))

df = pd.concat([champ_df, prem_df])
#df = df[(df['home_team'].isin(prem_teams_25)) | (df['away_team'].isin(prem_teams_25))]

df['match_date'] = pd.to_datetime(df['match_date'])
df = df[df["match_date"] > '2023-06-01']

print(df[["season", "match_date", "home_team", "away_team", "home_goals", "home_xgoals", "away_goals", "away_xgoals"]].tail())

        season match_date    home_team       away_team  home_goals  \
3895  20242025 2024-11-02       Wolves  Crystal Palace           2   
3896  20242025 2024-11-02  Bournemouth        Man City           2   
3897  20242025 2024-11-03   Man United         Chelsea           1   
3898  20242025 2024-11-03    Tottenham     Aston Villa           4   
3899  20242025 2024-11-04       Fulham       Brentford           2   

      home_xgoals  away_goals  away_xgoals  
3895      1.65018           2     2.648490  
3896      2.44930           1     2.352700  
3897      2.16050           1     1.210710  
3898      2.09751           1     2.487490  
3899      1.43017           1     0.739231  


In [2]:
# Apply averaged penalty when Championship match contains two promoted teams.

# Penalty xG = 0.665
# Penalty xGA = 1.465

# mask_both_prem = (df['division'] == 'Championship') & (df['home_team'].isin(prem_teams_25)) & (df['away_team'].isin(prem_teams_25))
# # #df.loc[mask_both_prem, 'home_goals'] *= 0.661
# # #df.loc[mask_both_prem, 'away_goals'] *= 0.661
# df.loc[mask_both_prem, 'home_xgoals'] *= 0.665
# df.loc[mask_both_prem, 'away_xgoals'] *= 0.655

# # Adjust performance penalty to championship games when one team is a promoted team.
# mask_home = (df['division'] == 'Championship') & (df['home_team'].isin(prem_teams_25)) & ~(df['away_team'].isin(prem_teams_25))
# # #df.loc[mask_home, 'home_goals'] *= 0.661
# # #df.loc[mask_home, 'away_goals'] *= 2.060
# df.loc[mask_home, 'home_xgoals'] *= 0.665
# df.loc[mask_home, 'away_xgoals'] *= 1.465

# mask_away = (df['division'] == 'Championship') & (df['away_team'].isin(prem_teams_25)) & ~(df['home_team'].isin(prem_teams_25))
# # #df.loc[mask_away, 'home_goals'] *= 2.060
# # #df.loc[mask_away, 'away_goals'] *= 0.661
# df.loc[mask_away, 'home_xgoals'] *= 1.465
# df.loc[mask_away, 'away_xgoals'] *= 0.665


# df[['home_goals', 'away_goals']] = df[['home_goals', 'away_goals']].round()


In [3]:
def decay(xi, t):
    return np.exp(-xi * t)

def rho_correction(goals_home, goals_away, home_exp, away_exp, rho):
    if goals_home == 0 and goals_away == 0:
        return 1 - (home_exp * away_exp * rho)
    elif goals_home == 0 and goals_away == 1:
        return 1 + (home_exp * rho)
    elif goals_home == 1 and goals_away == 0:
        return 1 + (away_exp * rho)
    elif goals_home == 1 and goals_away == 1:
        return 1 - rho
    else:
        return 1.0

In [4]:
def log_likelihood_xg(
    xG_home_observed,
    xG_away_observed,
    home_attack,
    home_defence,
    away_attack,
    away_defence,
    home_advantage,
    weight,
    sigma=1.0
):
    xG_expectation_home = np.exp(home_attack + away_defence + home_advantage)
    xG_expectation_away = np.exp(away_attack + home_defence)

    home_llk = norm.pdf(xG_home_observed, loc=xG_expectation_home, scale=sigma)
    away_llk = norm.pdf(xG_away_observed, loc=xG_expectation_away, scale=sigma)

    if xG_expectation_home < 0 or xG_expectation_away < 0:
        return 10000

    log_llk = weight * (np.log(home_llk) + np.log(away_llk))

    return -log_llk


def fit_xG_model(df, xi=0.0001, sigma=1.0):
    teams = np.sort(np.unique(np.concatenate([df["home_team"], df["away_team"]])))
    n_teams = len(teams)
    
    df["league_strength"] = df["division"].apply(lambda x: 1 if x == "Premier League" else 0.615) 
    df["days_since"] = (df["match_date"].max() - df["match_date"]).dt.days
    df["weight"] = decay(xi, df["days_since"]) * df["league_strength"]

    params = np.concatenate(
        (
            np.random.uniform(0.5, 1.5, (n_teams)),  # attack strength
            np.random.uniform(0, -1, (n_teams)),  # defence strength
            [0.25],  # home advantage
        )
    )

    bounds = [(None, None)] * (2 * n_teams) + [(None, None)]

    def _fit(params, df, teams, sigma):
        attack_params = dict(zip(teams, params[:n_teams]))
        defence_params = dict(zip(teams, params[n_teams : (2 * n_teams)]))
        home_advantage = params[-1]

        llk = list()
        for idx, row in df.iterrows():
            tmp = log_likelihood_xg(
                row["home_xgoals"],
                row["away_xgoals"],
                attack_params[row["home_team"]],
                defence_params[row["home_team"]],
                attack_params[row["away_team"]],
                defence_params[row["away_team"]],
                home_advantage,
                row["weight"],
                sigma=sigma
            )
            llk.append(tmp)

        return np.sum(llk)

    options = {
        "maxiter": 100,
        "disp": False,
    }

    constraints = [{"type": "eq", "fun": lambda x: sum(x[:n_teams]) - n_teams}]

    res = minimize(
        _fit,
        params,
        args=(df, teams, sigma),
        constraints=constraints,
        bounds=bounds,
        method='L-BFGS-B',
        options=options,
    )

    model_params = dict(
        zip(
            ["attack_" + team for team in teams]
            + ["defence_" + team for team in teams]
            + ["home_adv"],
            res["x"],
        )
    )

    # Prepare table data
    attack_values = [model_params[f'attack_{team}'] for team in teams]
    defence_values = [model_params[f'defence_{team}'] for team in teams]
    home_adv = model_params['home_adv']

    # Create DataFrame for attack and defense
    team_strength_df = pd.DataFrame({
        'Team': teams,
        'Attack Strength': attack_values,
        'Defense Strength': defence_values
    })

    # Add home advantage to the DataFrame
    additional_params_df = pd.DataFrame({
        'Parameter': ['Home Advantage'],
        'Value': [home_adv]
    })

    # Print DataFrames
    print("Team Strength (Attack and Defense):")
    print(team_strength_df)
    print("\nAdditional Parameters (Home Advantage):")
    print(additional_params_df)

    print("Log Likelihood: ", res["fun"])

    return model_params


xg_model_params = fit_xG_model(df, xi=0.001, sigma=1.0)

C:\Users\Owner\AppData\Local\Temp\ipykernel_15720\501073250.py:73: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  res = minimize(


Team Strength (Attack and Defense):
                Team  Attack Strength  Defense Strength
0            Arsenal         1.509828         -1.431580
1        Aston Villa         1.295942         -0.869098
2         Birmingham         0.396629         -0.445553
3          Blackburn         0.554267         -0.414938
4        Bournemouth         1.276624         -0.892261
5          Brentford         1.279380         -0.920628
6           Brighton         1.218058         -0.952144
7       Bristol City         0.491089         -0.535966
8            Burnley         0.716032         -0.714469
9            Cardiff         0.287172         -0.326115
10           Chelsea         1.489970         -0.913557
11          Coventry         0.630296         -0.559032
12    Crystal Palace         1.097772         -0.918901
13             Derby         0.298681         -0.414955
14           Everton         1.143015         -0.873880
15            Fulham         1.134157         -0.933127
16      Hudd

In [5]:
def log_likelihood_goals(
    goals_home_observed,
    goals_away_observed,
    home_attack,
    home_defence,
    away_attack,
    away_defence,
    home_advantage,
    rho,
    weight
):
    goal_expectation_home = np.exp(home_attack + away_defence + home_advantage)
    goal_expectation_away = np.exp(away_attack + home_defence)

    home_llk = poisson.pmf(goals_home_observed, goal_expectation_home)
    away_llk = poisson.pmf(goals_away_observed, goal_expectation_away)
    adj_llk = rho_correction(
        goals_home_observed,
        goals_away_observed,
        goal_expectation_home,
        goal_expectation_away,
        rho,
    )

    if goal_expectation_home < 0 or goal_expectation_away < 0 or adj_llk < 0:
        return 10000

    log_llk = weight * (np.log(home_llk) + np.log(away_llk) + np.log(adj_llk))

    return -log_llk

def fit_poisson_model(df, xi=0.0001):
    teams = np.sort(np.unique(np.concatenate([df["home_team"], df["away_team"]])))
    n_teams = len(teams)
    
    df["league_strength"] = df["division"].apply(lambda x: 1 if x == "Premier League" else 0.615) 
    df["days_since"] = (df["match_date"].max() - df["match_date"]).dt.days
    df["weight"] = decay(xi, df["days_since"]) * df["league_strength"]

    params = np.concatenate(
        (
            np.random.uniform(0.5, 1.5, (n_teams)),  # attack strength
            np.random.uniform(0, -1, (n_teams)),  # defence strength
            [0.25],  # home advantage
            [-0.1], # rho
        )
    )

    bounds = [(None, None)] * (2 * n_teams) + [(None, None), (-1, 1)]

    def _fit(params, df, teams):
        attack_params = dict(zip(teams, params[:n_teams]))
        defence_params = dict(zip(teams, params[n_teams : (2 * n_teams)]))
        home_advantage = params[-2]
        rho = params[-1]

        llk = list()
        for idx, row in df.iterrows():
            tmp = log_likelihood_goals(
                row["home_goals"],
                row["away_goals"],
                attack_params[row["home_team"]],
                defence_params[row["home_team"]],
                attack_params[row["away_team"]],
                defence_params[row["away_team"]],
                home_advantage,
                rho,
                row["weight"]
            )
            llk.append(tmp)

        return np.sum(llk)

    options = {
        "maxiter": 100,
        "disp": False,
    }

    constraints = [{"type": "eq", "fun": lambda x: sum(x[:n_teams]) - n_teams}]

    res = minimize(
        _fit,
        params,
        args=(df, teams),
        constraints=constraints,
        method='L-BFGS-B',
        options=options,
        bounds=bounds
    )

    model_params = dict(
        zip(
            ["attack_" + team for team in teams]
            + ["defence_" + team for team in teams]
            + ["home_adv", "rho"],
            res["x"],
        )
    )

    # Prepare table data
    attack_values = [model_params[f'attack_{team}'] for team in teams]
    defence_values = [model_params[f'defence_{team}'] for team in teams]
    home_adv = model_params['home_adv']
    rho = model_params['rho']

    # Create DataFrame for attack and defense
    team_strength_df = pd.DataFrame({
        'Team': teams,
        'Attack Strength': attack_values,
        'Defense Strength': defence_values
    })

    # Add home advantage and rho to the DataFrame
    additional_params_df = pd.DataFrame({
        'Parameter': ['Home Advantage', 'Rho'],
        'Value': [home_adv, rho]
    })

    # Print DataFrames
    print("Team Strength (Attack and Defense):")
    print(team_strength_df)
    print("\nAdditional Parameters (Home Advantage and Rho):")
    print(additional_params_df)

    print("Log Likelihood: ", res["fun"])

    return model_params

goals_model_params = fit_poisson_model(df, xi=0.001)

C:\Users\Owner\AppData\Local\Temp\ipykernel_15720\2509556092.py:81: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  res = minimize(


Team Strength (Attack and Defense):
                Team  Attack Strength  Defense Strength
0            Arsenal         1.548940         -1.522294
1        Aston Villa         1.412042         -0.849021
2         Birmingham         0.404300         -0.380432
3          Blackburn         0.565476         -0.342843
4        Bournemouth         1.122071         -0.888910
5          Brentford         1.217856         -0.761933
6           Brighton         1.165252         -0.907686
7       Bristol City         0.497276         -0.559341
8            Burnley         0.802300         -0.708600
9            Cardiff         0.440779         -0.287726
10           Chelsea         1.500288         -0.896239
11          Coventry         0.697191         -0.467911
12    Crystal Palace         1.070845         -0.962632
13             Derby         0.555334         -0.399302
14           Everton         0.782466         -0.992515
15            Fulham         1.125257         -0.911595
16      Hudd

In [6]:
def predict(params, home_team, away_team):
    home_attack = params["attack_" + home_team]
    home_defence = params["defence_" + home_team]
    away_attack = params["attack_" + away_team]
    away_defence = params["defence_" + away_team]
    home_advantage = params["home_adv"]
    rho = params.get("rho", 0)

    home_goal_expectation = np.exp(home_attack + away_defence + home_advantage)
    away_goal_expectation = np.exp(away_attack + home_defence)

    home_probs = poisson.pmf(range(10), home_goal_expectation)
    away_probs = poisson.pmf(range(10), away_goal_expectation)

    m = np.outer(home_probs, away_probs)

    m[0, 0] *= 1 - home_goal_expectation * away_goal_expectation * rho
    m[0, 1] *= 1 + home_goal_expectation * rho
    m[1, 0] *= 1 + away_goal_expectation * rho
    m[1, 1] *= 1 - rho    

    home = np.sum(np.tril(m, -1)) 
    draw = np.sum(np.diag(m)) 
    away = np.sum(np.triu(m, 1))

    total_prob = home + draw + away
    home /= total_prob
    draw /= total_prob
    away /= total_prob 

    # Calculate the probability of a clean sheet for the home team (away team scores 0)
    home_clean_sheet_prob = m[:, 0].sum() 

    # Calculate the probability of a clean sheet for the away team (home team scores 0)
    away_clean_sheet_prob = m[0, :].sum() 

    # Calculate the probability of the home team scoring 3 or more goals
    home_3_plus_goals_prob = home_probs[3:].sum() 

    # Calculate the probability of the away team scoring 3 or more goals
    away_3_plus_goals_prob = away_probs[3:].sum() 


    return {
        "home_win_prob": home.round(2),
        "draw_prob": draw.round(2),
        "away_win_prob": away.round(2),
        "home_clean_sheet_prob": home_clean_sheet_prob.round(2),
        "away_clean_sheet_prob": away_clean_sheet_prob.round(2),
        "home_goal_expectation": home_goal_expectation.round(2),
        "away_goal_expectation": away_goal_expectation.round(2),
        "home_3_plus_goals_prob": home_3_plus_goals_prob.round(2),
        "away_3_plus_goals_prob": away_3_plus_goals_prob.round(2)
    }

In [7]:
# Initialize lists to store predictions
xg_home_preds = []
xg_away_preds = []
goals_home_preds = []
goals_away_preds = []

# Iterate through the games in your dataset
for idx, row in df.iterrows():
    xg_pred = predict(xg_model_params, row['home_team'], row['away_team'])
    goals_pred = predict(goals_model_params, row['home_team'], row['away_team'])

    xg_home_preds.append(xg_pred['home_goal_expectation'])
    xg_away_preds.append(xg_pred['away_goal_expectation'])
    goals_home_preds.append(goals_pred['home_goal_expectation'])
    goals_away_preds.append(goals_pred['away_goal_expectation'])

# Add these predictions back to the DataFrame
df['xg_home_pred'] = xg_home_preds
df['xg_away_pred'] = xg_away_preds
df['goals_home_pred'] = goals_home_preds
df['goals_away_pred'] = goals_away_preds

# Calculate absolute errors for xG predictions
df['xg_home_error'] = abs(df['home_xgoals'] - df['xg_home_pred'])
df['xg_away_error'] = abs(df['away_xgoals'] - df['xg_away_pred'])

# Calculate absolute errors for goals predictions
df['goals_home_error'] = abs(df['home_goals'] - df['goals_home_pred'])
df['goals_away_error'] = abs(df['away_goals'] - df['goals_away_pred'])


# Calculate MAE for xG predictions
xg_mae_home = df['xg_home_error'].mean()
xg_mae_away = df['xg_away_error'].mean()

# Calculate MAE for goals predictions
goals_mae_home = df['goals_home_error'].mean()
goals_mae_away = df['goals_away_error'].mean()

# Combine MAEs for home and away
xg_mae_total = (xg_mae_home + xg_mae_away) / 2
goals_mae_total = (goals_mae_home + goals_mae_away) / 2

print(f"xG MAE (home): {xg_mae_home}")
print(f"xG MAE (away): {xg_mae_away}")
print(f"xG MAE (total): {xg_mae_total}")

print(f"Goals MAE (home): {goals_mae_home}")
print(f"Goals MAE (away): {goals_mae_away}")
print(f"Goals MAE (total): {goals_mae_total}")

xG MAE (home): 0.5829920296646603
xG MAE (away): 0.5236610533104041
xG MAE (total): 0.5533265414875322
Goals MAE (home): 0.9440326741186585
Goals MAE (away): 0.8393809114359416
Goals MAE (total): 0.8917067927773001


In [10]:
fixtures = [
    ("Brentford", "Bournemouth"),
    ("Crystal Palace", "Fulham"),
    ("West Ham", "Everton"),
    ("Wolves", "Southampton"),
    ("Brighton", "Man City"),
    ("Liverpool", "Aston Villa"),
    ("Man United", "Leicester"),
    ("Nott'm Forest", "Newcastle"),
    ("Tottenham", "Ipswich"),
    ("Chelsea", "Arsenal")
]

results = []

for fixture in fixtures:
    home_team, away_team = fixture
    
    xg_preds = predict(xg_model_params, home_team, away_team)
    goals_preds = predict(goals_model_params, home_team, away_team)
    
    weighted_preds = {}
    
    for key in xg_preds.keys():
        weighted_value = (0.7 * xg_preds[key]) + (0.3 * goals_preds[key])
        weighted_preds[key] = weighted_value
    
    results.append({
        'home_team': home_team,
        'away_team': away_team,
        **weighted_preds
    })

results_df = pd.DataFrame(results)


# Display the results DataFrame
display(results_df)

,home_team,away_team,home_win_prob,draw_prob,away_win_prob,home_clean_sheet_prob,away_clean_sheet_prob,home_goal_expectation,away_goal_expectation,home_3_plus_goals_prob,away_3_plus_goals_prob
0,Brentford,Bournemouth,0.475,0.226,0.299,0.240,0.159,1.853,1.430,0.285,0.170
1,Crystal Palace,Fulham,0.437,0.253,0.307,0.296,0.223,1.509,1.222,0.194,0.127
2,West Ham,Everton,0.432,0.239,0.329,0.239,0.185,1.678,1.453,0.239,0.184
3,Wolves,Southampton,0.551,0.214,0.235,0.296,0.138,1.987,1.215,0.321,0.127
4,Brighton,Man City,0.222,0.224,0.554,0.151,0.328,1.109,1.876,0.101,0.288
5,Liverpool,Aston Villa,0.712,0.159,0.129,0.340,0.069,2.692,1.084,0.503,0.097
6,Man United,Leicester,0.645,0.184,0.171,0.304,0.096,2.448,1.188,0.439,0.115
7,Nott'm Forest,Newcastle,0.376,0.240,0.384,0.211,0.218,1.528,1.542,0.202,0.202
8,Tottenham,Ipswich,0.797,0.123,0.083,0.378,0.040,3.136,0.970,0.603,0.076
9,Chelsea,Arsenal,0.275,0.227,0.498,0.157,0.265,1.322,1.850,0.148,0.279


In [11]:
results_df.to_csv("predictions/ensmeble_gw11_preds.csv")